In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [24]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [25]:
fake=pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')


In [12]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [26]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [27]:
fake['class']=0
true['class']=1

In [28]:
data=pd.concat([fake,true],axis=0)

In [29]:
data.sample(10)

,title,text,subject,date,class
4901,Rachel Maddow Asked Trump’s Campaign Manager ...,By now most of us know that anytime a member o...,News,"August 25, 2016",0
17433,FACTBOX - German coalition watch: Agreeing on ...,BERLIN (Reuters) - Chancellor Angela Merkel wo...,worldnews,"October 15, 2017",1
11068,Biden sees growing progress in battle against ...,BALTIMORE (Reuters) - The U.S.-led battle agai...,politicsNews,"January 28, 2016",1
15613,Future Saudi king tightens grip on power with ...,RIYADH (Reuters) - Saudi Arabia s future king ...,worldnews,"November 5, 2017",1
2547,Schumer says Senate Democrats will not block v...,WASHINGTON (Reuters) - U.S. Senate Democratic ...,politicsNews,"July 25, 2017",1
36,"Congress votes to avert shutdown, sends Trump ...",WASHINGTON (Reuters) - The U.S. Congress on Th...,politicsNews,"December 21, 2017",1
11919,Croatia and Slovenia fail to move forward in t...,ZAGREB (Reuters) - Croatia and Slovenia failed...,worldnews,"December 19, 2017",1
6181,White Students Rape Black Teen With Coat Hang...,A federal lawsuit filed against officials at D...,News,"May 25, 2016",0
19352,UK PM May says Brexit transition period to las...,"FLORENCE, Italy (Reuters) - Prime Minister The...",worldnews,"September 22, 2017",1
4438,Neo-Nazis Are Flocking To Trump’s Campaign An...,The Nazis see Donald Trump as their pathway ba...,News,"September 29, 2016",0


In [30]:
data=data.drop(["title","subject","date"],axis=1)

In [31]:
data.reset_index(inplace=True)

In [32]:
data.drop(['index'],axis=1,inplace=True)

In [20]:
data.sample(5)

,text,class
11621,Sen. John McCain(R-Ariz.) dug himself into a h...,0
27266,WASHINGTON (Reuters) - U.S. Senator Mark Warne...,1
817,Reaching an almost all-time low even by their ...,0
31397,"HENDERSON, Nev. (Reuters) - U.S. Republican pr...",1
27722,(Reuters) - The Trump administration on Friday...,1


In [35]:
def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]',"",text)
    text=re.sub("\\W"," ",text)
    text=re.sub("https?:://\S+|www\. \S+","",text)
    text=re.sub("<.*?>+","",text)
    text=re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text=re.sub("\n","",text)
    text=re.sub("\w*\d\w*","",text)
    return text
    

In [36]:
data["text"]=data["text"].apply(clean_text)

In [37]:
x=data["text"]
y=data["class"]
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [45]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [41]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [42]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [43]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [47]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']